In [1]:
%reload_ext autoreload
%autoreload 2

import sys
import os
import time

import numpy as np
import bloscpack as bp
import matplotlib.pyplot as plt
%matplotlib inline
import vtk
from vtk.util import numpy_support

sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *
from annotation_utilities import *
from metadata import *
from vis3d_utilities import *

Setting environment for Local Macbook Pro
/home/yuncong/CSHL_data_processed/MD585/MD585_prep2_thumbnail/MD585-N16-2015.07.16-20.32.33_MD585_2_0047_prep2_thumbnail.tif
/home/yuncong/CSHL_data_processed/MD589/MD589_prep2_thumbnail/MD589-N16-2015.07.30-17.03.43_MD589_3_0048_prep2_thumbnail.tif
/home/yuncong/CSHL_data_processed/MD590/MD590_prep2_thumbnail/MD590-IHC17-2015.08.10-19.09.09_MD590_2_0050_prep2_thumbnail.tif
/home/yuncong/CSHL_data_processed/MD591/MD591_prep2_thumbnail/MD591-IHC17-2015.08.28-04.23.47_MD591_1_0049_prep2_thumbnail.tif
/home/yuncong/CSHL_data_processed/MD592/MD592_prep2_thumbnail/MD592-N16-2015.08.22-00.50.39_MD592_2_0047_prep2_thumbnail.tif
/home/yuncong/CSHL_data_processed/MD593/MD593_prep2_thumbnail/MD593-N15-2015.08.21-16.53.02_MD593_3_0045_prep2_thumbnail.tif
/home/yuncong/CSHL_data_processed/MD594/MD594_prep2_thumbnail/MD594-IHC16-2015.08.26-16.11.01_MD594_1_0046_prep2_thumbnail.tif
/home/yuncong/CSHL_data_processed/MD595/MD595_prep2_thumbnail/MD595-IHC12-201

Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Version Ntb is not available. Instead, load lossless and take the blue channel...


/home/yuncong/CSHL_data_processed/ChatCryoJane201710/ChatCryoJane201710_prep2_thumbnail/ChatCryoJane201710_slide1-S1_prep2_thumbnail.tif


Not using image_cache.


In [105]:
brain_spec = dict(name='MD589', 
                  resolution='10.0um', 
                  vol_type='annotationAsScore', 
                  prep_id=None,
                  detector_id=None,
                  structure=None)

stack = brain_spec['name']

# Construct annotation volumes from annotation contours

In [66]:
contour_df = DataManager.load_annotation_v4(stack=stack, by_human=True, timestamp='latest', suffix='contours')
contour_df = convert_annotation_v3_original_to_aligned(contour_df, stack=stack)
# contour_df vertices wrt to aligned uncropped lossless image aka "wholebrain"

aws s3 cp --recursive "s3://mousebrainatlas-data/CSHL_labelings_v3/MD594" "/home/yuncong/CSHL_labelings_v3/MD594" --exclude "*" --include "*contours*"
latest timestamp:  07302017183604


In [67]:
for name_s in all_known_structures_sided:
# for name_s in ['7N_L']:

    try:

        name_u, side = parse_label(name_s, singular_as_s=True)[:2]

        contours_xysec_wrt_wholebrain_rawResol = [[(x, y, cnt['section']) for x, y in cnt['vertices']]
         for cid, cnt in contour_df[(contour_df['name'] == name_u) & (contour_df['side'] == side)].iterrows()]

        contours_xyz_wrt_wholebrain_volResol = \
        [convert_frame_and_resolution(cnt_xysec,
                                      in_wrt='wholebrain', in_resolution='image_image_section',
                                      out_wrt='wholebrain', out_resolution='volume',
                                     image_resolution='raw', stack=stack,
                                     volume_resolution_um=10.)
         for cnt_xysec in contours_xysec_wrt_wholebrain_rawResol]

        volume_volResol, bbox_wrt_wholebrain_volResol = \
        interpolate_contours_to_volume(contours_xyz=contours_xyz_wrt_wholebrain_volResol, 
                                       interpolation_direction='z',
                                      return_origin_instead_of_bbox=False)


        # Save

        ann_vol_fp = DataManager.get_original_volume_filepath_v2(stack_spec=brain_spec, structure=name_s)
        create_parent_dir_if_not_exists(ann_vol_fp)
        bp.pack_ndarray_file(volume_volResol, ann_vol_fp)
        upload_to_s3(ann_vol_fp)

        ann_bbox_fp = DataManager.get_original_volume_bbox_filepath_v2(stack_spec=brain_spec, structure=name_s)
        np.savetxt(ann_bbox_fp, bbox_wrt_wholebrain_volResol)
        upload_to_s3(ann_bbox_fp)
        
    except:
        sys.stderr.write("Error reconstructing %s\n" % name_s)
        continue

aws s3 cp "/media/yuncong/YuncongPublic/CSHL_volumes/MD594/MD594_10.0um_annotationAsScoreVolume/score_volumes/MD594_10.0um_annotationAsScoreVolume_5N_L.bp" "s3://mousebrainatlas-data/CSHL_volumes/MD594/MD594_10.0um_annotationAsScoreVolume/score_volumes/MD594_10.0um_annotationAsScoreVolume_5N_L.bp"
aws s3 cp "/media/yuncong/YuncongPublic/CSHL_volumes/MD594/MD594_10.0um_annotationAsScoreVolume/score_volumes/MD594_10.0um_annotationAsScoreVolume_5N_L_bbox_wrt_wholebrain.txt" "s3://mousebrainatlas-data/CSHL_volumes/MD594/MD594_10.0um_annotationAsScoreVolume/score_volumes/MD594_10.0um_annotationAsScoreVolume_5N_L_bbox_wrt_wholebrain.txt"
aws s3 cp "/media/yuncong/YuncongPublic/CSHL_volumes/MD594/MD594_10.0um_annotationAsScoreVolume/score_volumes/MD594_10.0um_annotationAsScoreVolume_5N_R.bp" "s3://mousebrainatlas-data/CSHL_volumes/MD594/MD594_10.0um_annotationAsScoreVolume/score_volumes/MD594_10.0um_annotationAsScoreVolume_5N_R.bp"
aws s3 cp "/media/yuncong/YuncongPublic/CSHL_volumes/MD594/MD

aws s3 cp "/media/yuncong/YuncongPublic/CSHL_volumes/MD594/MD594_10.0um_annotationAsScoreVolume/score_volumes/MD594_10.0um_annotationAsScoreVolume_LRt_R.bp" "s3://mousebrainatlas-data/CSHL_volumes/MD594/MD594_10.0um_annotationAsScoreVolume/score_volumes/MD594_10.0um_annotationAsScoreVolume_LRt_R.bp"
aws s3 cp "/media/yuncong/YuncongPublic/CSHL_volumes/MD594/MD594_10.0um_annotationAsScoreVolume/score_volumes/MD594_10.0um_annotationAsScoreVolume_LRt_R_bbox_wrt_wholebrain.txt" "s3://mousebrainatlas-data/CSHL_volumes/MD594/MD594_10.0um_annotationAsScoreVolume/score_volumes/MD594_10.0um_annotationAsScoreVolume_LRt_R_bbox_wrt_wholebrain.txt"
aws s3 cp "/media/yuncong/YuncongPublic/CSHL_volumes/MD594/MD594_10.0um_annotationAsScoreVolume/score_volumes/MD594_10.0um_annotationAsScoreVolume_Pn_L.bp" "s3://mousebrainatlas-data/CSHL_volumes/MD594/MD594_10.0um_annotationAsScoreVolume/score_volumes/MD594_10.0um_annotationAsScoreVolume_Pn_L.bp"
aws s3 cp "/media/yuncong/YuncongPublic/CSHL_volumes/MD59

aws s3 cp "/media/yuncong/YuncongPublic/CSHL_volumes/MD594/MD594_10.0um_annotationAsScoreVolume/score_volumes/MD594_10.0um_annotationAsScoreVolume_3N_L.bp" "s3://mousebrainatlas-data/CSHL_volumes/MD594/MD594_10.0um_annotationAsScoreVolume/score_volumes/MD594_10.0um_annotationAsScoreVolume_3N_L.bp"
aws s3 cp "/media/yuncong/YuncongPublic/CSHL_volumes/MD594/MD594_10.0um_annotationAsScoreVolume/score_volumes/MD594_10.0um_annotationAsScoreVolume_3N_L_bbox_wrt_wholebrain.txt" "s3://mousebrainatlas-data/CSHL_volumes/MD594/MD594_10.0um_annotationAsScoreVolume/score_volumes/MD594_10.0um_annotationAsScoreVolume_3N_L_bbox_wrt_wholebrain.txt"
aws s3 cp "/media/yuncong/YuncongPublic/CSHL_volumes/MD594/MD594_10.0um_annotationAsScoreVolume/score_volumes/MD594_10.0um_annotationAsScoreVolume_3N_R.bp" "s3://mousebrainatlas-data/CSHL_volumes/MD594/MD594_10.0um_annotationAsScoreVolume/score_volumes/MD594_10.0um_annotationAsScoreVolume_3N_R.bp"
aws s3 cp "/media/yuncong/YuncongPublic/CSHL_volumes/MD594/MD

aws s3 cp "/media/yuncong/YuncongPublic/CSHL_volumes/MD594/MD594_10.0um_annotationAsScoreVolume/score_volumes/MD594_10.0um_annotationAsScoreVolume_10N_R.bp" "s3://mousebrainatlas-data/CSHL_volumes/MD594/MD594_10.0um_annotationAsScoreVolume/score_volumes/MD594_10.0um_annotationAsScoreVolume_10N_R.bp"
aws s3 cp "/media/yuncong/YuncongPublic/CSHL_volumes/MD594/MD594_10.0um_annotationAsScoreVolume/score_volumes/MD594_10.0um_annotationAsScoreVolume_10N_R_bbox_wrt_wholebrain.txt" "s3://mousebrainatlas-data/CSHL_volumes/MD594/MD594_10.0um_annotationAsScoreVolume/score_volumes/MD594_10.0um_annotationAsScoreVolume_10N_R_bbox_wrt_wholebrain.txt"
aws s3 cp "/media/yuncong/YuncongPublic/CSHL_volumes/MD594/MD594_10.0um_annotationAsScoreVolume/score_volumes/MD594_10.0um_annotationAsScoreVolume_VCA_L.bp" "s3://mousebrainatlas-data/CSHL_volumes/MD594/MD594_10.0um_annotationAsScoreVolume/score_volumes/MD594_10.0um_annotationAsScoreVolume_VCA_L.bp"
aws s3 cp "/media/yuncong/YuncongPublic/CSHL_volumes/MD

# Load annotation volumes and convert to meshes

In [68]:
for name_s in all_known_structures_sided:
# for name_s in ['7N_L']:

    try:
        vol, origin_wrt_wholebrain_volResol = DataManager.load_original_volume_v2(stack_spec=brain_spec,
                                                structure=name_s, return_origin_instead_of_bbox=True)
    except:
        sys.stderr.write("Error loading %s\n" % name_s)
        continue
        
    t = time.time()
    mesh = volume_to_polydata(vol, origin=origin_wrt_wholebrain_volResol, num_simplify_iter=3, smooth=True)

    sys.stderr.write('time: %.2f\n' % (time.time() - t) ) # ~ 150s for outerContour; 93s for sp5

    mesh_fp = DataManager.get_mesh_filepath_v2(brain_spec=brain_spec, structure=name_s)
    create_parent_dir_if_not_exists(mesh_fp)
    save_mesh_stl(mesh, mesh_fp)
    upload_to_s3(mesh_fp)

marching cube: 0.05 seconds
mesh_to_polydata: 0.03 seconds
simplify 0 @ 15713: 0.20 seconds
simplify 1 @ 3144: 0.18 seconds
simplify 2 @ 630: 0.04 seconds
time: 0.51


aws s3 cp "/home/yuncong/CSHL_meshes/MD594/MD594_10.0um_annotationAsScoreVolume/MD594_10.0um_annotationAsScoreVolume_5N_L.stl" "s3://mousebrainatlas-data/CSHL_meshes/MD594/MD594_10.0um_annotationAsScoreVolume/MD594_10.0um_annotationAsScoreVolume_5N_L.stl"


marching cube: 0.04 seconds
mesh_to_polydata: 0.02 seconds
simplify 0 @ 13688: 0.14 seconds
simplify 1 @ 2739: 0.11 seconds
simplify 2 @ 549: 0.03 seconds
time: 0.34


aws s3 cp "/home/yuncong/CSHL_meshes/MD594/MD594_10.0um_annotationAsScoreVolume/MD594_10.0um_annotationAsScoreVolume_5N_R.stl" "s3://mousebrainatlas-data/CSHL_meshes/MD594/MD594_10.0um_annotationAsScoreVolume/MD594_10.0um_annotationAsScoreVolume_5N_R.stl"


marching cube: 0.00 seconds
mesh_to_polydata: 0.00 seconds
simplify 0 @ 1917: 0.02 seconds
simplify 1 @ 385: 0.01 seconds
simplify 2 @ 78: 0.00 seconds
time: 0.05


...

aws s3 cp "/home/yuncong/CSHL_meshes/MD594/MD594_10.0um_annotationAsScoreVolume/MD594_10.0um_annotationAsScoreVolume_RtTg.stl" "s3://mousebrainatlas-data/CSHL_meshes/MD594/MD594_10.0um_annotationAsScoreVolume/MD594_10.0um_annotationAsScoreVolume_RtTg.stl"


marching cube: 2.24 seconds
mesh_to_polydata: 0.33 seconds
simplify 0 @ 279025: 2.70 seconds
simplify 1 @ 55807: 4.37 seconds
simplify 2 @ 11162: 1.12 seconds
time: 10.87


aws s3 cp "/home/yuncong/CSHL_meshes/MD594/MD594_10.0um_annotationAsScoreVolume/MD594_10.0um_annotationAsScoreVolume_SC.stl" "s3://mousebrainatlas-data/CSHL_meshes/MD594/MD594_10.0um_annotationAsScoreVolume/MD594_10.0um_annotationAsScoreVolume_SC.stl"


marching cube: 0.95 seconds
mesh_to_polydata: 0.19 seconds
simplify 0 @ 145616: 2.12 seconds
simplify 1 @ 29125: 1.82 seconds
simplify 2 @ 5826: 0.37 seconds
time: 5.49


aws s3 cp "/home/yuncong/CSHL_meshes/MD594/MD594_10.0um_annotationAsScoreVolume/MD594_10.0um_annotationAsScoreVolume_IC.stl" "s3://mousebrainatlas-data/CSHL_meshes/MD594/MD594_10.0um_annotationAsScoreVolume/MD594_10.0um_annotationAsScoreVolume_IC.stl"


# Load meshes

In [106]:
meshes_wrt_wholebrain_volResol = DataManager.load_meshes_v2(brain_spec=brain_spec, sided=True)

In [107]:
mesh_actors_wrt_wholebrain_volResol = [actor_mesh(mesh, np.array(name_unsided_to_color[convert_to_original_name(name_s)])/255., 
                          wireframe=False, opacity=1. if name_s in ['7N_L', '7N_R'] else 0.05) 
               for name_s, mesh in meshes_wrt_wholebrain_volResol.iteritems()]

In [108]:
shell_polydata_wrt_wholebrain_volResol = DataManager.load_mesh_v2(brain_spec=brain_spec, structure='shell')
shell_actor_wrt_wholebrain_volResol = actor_mesh(shell_polydata_wrt_wholebrain_volResol, (1,1,1), opacity=.1, wireframe=False)

In [109]:
launch_vtk(mesh_actors_wrt_wholebrain_volResol + [shell_actor_wrt_wholebrain_volResol], 
           view_up=[0.056929, -0.993415, 0.099426],
          focal=[874.247985, 349.971904, 38.088624],
          position=[-165.978799, 148.917178, -1375.141280])

In [ ]:
# Export to rgb PLY file (STL file can not have color info)
# https://github.com/Kitware/VTK/blob/master/IO/PLY/Testing/Python/TestPLYReadWrite.py#L31
# http://www.vtk.org/doc/nightly/html/classvtkPLYWriter.html#aa7f0bdbb2decdc7a7360a890a6c10e8b

for name_s, polydata in polydata_list.iteritems():
    fn = os.path.join(mesh_rootdir, stack, "%(stack)s_down32_%(name)s_smoothed.ply" % {'stack': stack, 'name':name_s})
    save_mesh(polydata, fn, color=(colors[convert_name_to_unsided(name_s)]*255).astype(np.int))